<a href="https://colab.research.google.com/github/KKawaguchi1219/tf-idf_test/blob/main/tf_idf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**pip install and test**

In [1]:
!pip install mecab-python3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 581.7/581.7 kB 6.4 MB/s eta 0:00:00


In [2]:
!pip install unidic-lite

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 MB 16.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for unidic-lite: filename=unidic_lite-1.0.8-py3-none-any.whl size=47658816 sha256=8b95748befe61b1ff439be3f13cd59e24b57fece943e85a22b5e3143a7fd9911
  Stored in directory: /root/.cache/pip/wheels/89/e8/68/f9ac36b8cc6c8b3c96888cd57434abed96595d444f42243853
Successfully built unidic-lite


In [15]:
import MeCab

text='餃子は餃子が大好きです!'
wakati = MeCab.Tagger("-Owakati")
words=wakati.parse(text).split()

In [44]:
print(words)
print(words.count('餃子'))

['餃子', 'は', '餃子', 'が', '大好き', 'です', '!']
2


In [7]:
tagger = MeCab.Tagger()
print(tagger.parse(text))

餃子	ギョーザ	ギョーザ	ギョーザ	名詞-普通名詞-一般			0
が	ガ	ガ	が	助詞-格助詞			
大好き	ダイスキ	ダイスキ	大好き	形状詞-一般			1
です	デス	デス	です	助動詞	助動詞-デス	終止形-一般	
!			！	補助記号-句点			
EOS



In [43]:
import re

doc='I waana love you!'
hoge=[]
print(doc)
doc=re.sub('[.|?|!]', '', doc.lower())
doc=wakati.parse(doc).split()
hoge += doc
print(hoge)

I waana love you!
['i', 'waana', 'love', 'you']


**TF**

In [23]:
import MeCab
from collections import defaultdict

def tf_test(document):
    wakati = MeCab.Tagger("-Owakati")
    words=wakati.parse(document).split()
    words_freq=defaultdict(int)
    words_tf=defaultdict(int)
    for word in words:
        words_freq[word]+=1
        words_tf[word]=words_freq[word]/len(words)
    return len(words), words_tf

In [24]:
import MeCab

document='餃子は餃子が好きかもしれません。'
num, test_tf=tf_test(document=document)
print(f'単語数:{num}')
print(test_tf)

単語数:11
defaultdict(<class 'int'>, {'餃子': 0.18181818181818182, 'は': 0.09090909090909091, 'が': 0.09090909090909091, '好き': 0.09090909090909091, 'か': 0.09090909090909091, 'も': 0.09090909090909091, 'しれ': 0.09090909090909091, 'ませ': 0.09090909090909091, 'ん': 0.09090909090909091, '。': 0.09090909090909091})


**IDF**

In [50]:
import MeCab
import re
from collections import defaultdict
import numpy as np

def idf_test(documents):
    n=len(documents)

    documents = [re.sub('[、|。|!|?]', '', doc.lower()) for doc in documents]

    words_num=[]
    words_idf=defaultdict(int)

    for doc in documents:
        wakati = MeCab.Tagger("-Owakati")
        # 分かち書きした文章を集合型に直し、リストに格納する。
        # リストに格納される単語tに1文書中の重複はないため、リスト中の単語tの重複の数は単語tが出現する文書数dfと等価になるﾊｽﾞである。
        words=set(wakati.parse(doc).split())
        words_num += words

    for word in words_num:
        if word in words_idf.keys():
            continue
        df=words_num.count(word)
        # 各単語のidfを計算
        words_idf[word]=np.log(n/df)

    return words_idf

In [51]:
documents=[
    '餃子は好きです。',
    'あまり餃子が好きではありません。',
    '小籠包の方が好きです。',
    '私は私の彼女の作った餃子がいちばん好きです。'
]

idf=idf_test(documents)
idf

defaultdict(int,
            {'餃子': 0.28768207245178085,
             'です': 0.28768207245178085,
             '好き': 0.0,
             'は': 0.28768207245178085,
             'あまり': 1.3862943611198906,
             'あり': 1.3862943611198906,
             'で': 1.3862943611198906,
             'ん': 1.3862943611198906,
             'が': 0.28768207245178085,
             'ませ': 1.3862943611198906,
             '方': 1.3862943611198906,
             'の': 0.6931471805599453,
             '包': 1.3862943611198906,
             '小籠': 1.3862943611198906,
             '彼女': 1.3862943611198906,
             '作っ': 1.3862943611198906,
             'いちばん': 1.3862943611198906,
             '私': 1.3862943611198906,
             'た': 1.3862943611198906})